In [1]:
def func(x,
         a,
         b,
         c):
    return a * np.exp(-(x - b) ** 2 / c ** 2)


def evaluate_bell_curve_error(
    peak_idxs,
    start_idxs,
    end_idxs,
    signal,
    fs
):

    """This function calculates the bell-curve error of signal peaks

    :param signal: filtered signal
    :type signal: ~numpy.ndarray
    :param peak_idxs: list of peak indices
    :type peak_idxs: ~numpy.ndarray
    :param start_idxs: list of onsets indices
    :type start_idxs: ~numpy.ndarray
    :param end_idxs: list of offsets indices
    :type end_idxs: ~numpy.ndarray
    :param fs: sample rate
    :type fs: int
    :returns:ETP_bell_error
    :rtype: ~numpy.ndarray
    """

    t = np.array([i / fs for i in range(len(signal))])

    ETP_bell_error = np.zeros((len(peak_idxs),))
    Y_min = np.zeros((len(peak_idxs),))

    for idx in range(len(peak_idxs)):
        start_i = int(start_idxs[idx])
        end_i = int(end_idxs[idx])

        baseline_start_i = max(0, int(peak_idxs[idx]) - 5 * fs)
        baseline_end_i = min(len(signal) - 1, int(peak_idxs[idx]) + 5 * fs)
        Y_min[idx] = np.min(signal[baseline_start_i:baseline_end_i])

        if end_i - start_i < 3:
            plus_idx = 3 - (end_i - start_i)
        else:
            plus_idx = 0

        x_data = t[start_i:end_i + 1]
        y_data = signal[start_i:end_i + 1] - Y_min[idx]

        if np.any(np.isnan(x_data)) or np.any(np.isnan(y_data)) or np.any(np.isinf(x_data)) or np.any(np.isinf(y_data)):
            print(f"NaNs or Infs detected in data for peak index {idx}.Skipping this peak.")
            ETP_bell_error[idx] = np.nan
            continue

        try:
            popt, _ = curve_fit(
                func, x_data, y_data,
                bounds=([0., t[int(peak_idxs[idx])] - 0.5, 0.],
                        [60., t[int(peak_idxs[idx])] + 0.5, 0.5])
            )
        except RuntimeError as e:
            print(f"Curve fitting failed for peak index {idx} with error: {e}")
            ETP_bell_error[idx] = np.nan
            continue

        ETP_bell_error[idx] = np.trapz(
            np.sqrt((signal[start_i:end_i + 1] - (
                func(t[start_i:end_i + 1], *popt) + Y_min[idx])) ** 2),
            dx=1 / fs
        )

    return ETP_bell_error

In [2]:
import unittest
import numpy as np

class TestQualityAssessment(unittest.TestCase):

    def test_evaluate_bell_curve_error(self):
        self.fs = 1000
        self.duration = 60
        self.frequency = 12 / self.duration
        self.t = np.linspace(0, self.duration, int(self.duration * self.fs))
        self.signal = np.sin(2 * np.pi * self.frequency * self.t)
        self.peak_idxs = np.arange(self.fs // (2 * self.frequency), len(self.t),
                                  self.fs // self.frequency)
        self.start_idxs = self.peak_idxs - (self.fs // (2 * self.frequency))
        self.end_idxs = self.peak_idxs + (self.fs // (2 * self.frequency))

        # Ensure indices are within bounds
        self.start_idxs = self.start_idxs[self.start_idxs >= 0]
        self.end_idxs = self.end_idxs[self.end_idxs < len(self.signal)]
        self.peak_idxs = self.peak_idxs[:len(self.start_idxs)]

        # Add assertions or checks to verify the correctness of your code
        self.assertTrue(len(self.peak_idxs) > 0, "No peaks found")
        self.assertTrue(np.all(self.start_idxs >= 0), "Start indices out of bounds")
        self.assertTrue(np.all(self.end_idxs < len(self.signal)), "End indices out of bounds")

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)


.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK
